In [75]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [76]:
data = pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId','Surname'], axis=1)

In [77]:

LabelEncoder_gender = LabelEncoder()
data['Gender'] = LabelEncoder_gender.fit_transform(data['Gender'])

OneHotEncoder_geo = OneHotEncoder()
geo_encoded = OneHotEncoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = OneHotEncoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']
                                              

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [79]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(LabelEncoder_gender, file)

with open('OneHotEncoder_geo.pkl', 'wb') as file:
    pickle.dump(OneHotEncoder_geo, file)
    
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)


## Define a Function to create the model and try different paramters
### (KerasClassifier)


In [80]:
def create_model(neurons = 32, layers = 1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu',input_shape =(X_train.shape[1],)))
    
    for _ in range(layers-1):
        model.add(Dense(neurons, activation='relu'))
    
    model.add(Dense(1 , activation='sigmoid'))
    
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model
    

In [81]:
model = KerasClassifier(layers=1, neurons= 32, build_fn = create_model, verbose=1)

In [82]:
param_grid = {
    'neurons': [16,32,64,128],
    'layers': [1,2],
    'epochs': [50,100]
}


In [83]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs = -1, cv=3, verbose=1)
grid_result = grid.fit(X_train, y_train)

print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-pa

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 907us/step - accuracy: 0.4994 - loss: 0.7722  
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 841us/step - accuracy: 0.6136 - loss: 0.6568
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 935us/step - accuracy: 0.7403 - loss: 0.5874
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 610us/step - accuracy: 0.7533 - loss: 0.5375
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 817us/step - accuracy: 0.7867 - loss: 0.5133
Epoch 2/50
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 657us/step - accuracy: 0.6018 - loss: 0.6409
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6167 - loss: 0.6342
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7139 - loss: 0.5692
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step - accuracy: 0.7971 - loss: 0.4828
Epoch 3/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 683us/step - accuracy: 0.8009 - loss: 0.4861
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 503us/step - 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-pa

84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 1/50
Epoch 1/50


...

167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step - accuracy: 0.8660 - loss: 0.3221
Epoch 10/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 795us/step - accuracy: 0.8633 - loss: 0.3329
Epoch 9/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step - accuracy: 0.8600 - loss: 0.3299
Epoch 16/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step - accuracy: 0.8680 - loss: 0.3227
Epoch 16/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
  1/167 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.8438 - loss: 0.3800Epoch 1/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 689us/step - accuracy: 0.8656 - loss: 0.3234
Epoch 11/100


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step - accuracy: 0.8644 - loss: 0.3278
Epoch 17/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step - accuracy: 0.8639 - loss: 0.3175
Epoch 17/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step - accuracy: 0.8588 - loss: 0.3235
Epoch 12/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 682us/step - accuracy: 0.8709 - loss: 0.3165
Epoch 18/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 867us/step - accuracy: 0.7954 - loss: 0.4876
  1/167 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.8750 - loss: 0.2458Epoch 2/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 543us/step - accuracy: 0.8694 - loss: 0.3121
Epoch 10/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8722 - loss: 0.3179 
Epoch 18/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 880us/step - accuracy: 0.8266 - loss: 0.4162
Epoch 3/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8760 - loss: 0.3119
Epoch 19/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8642 - loss: 0.3274
Epoch 13/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 342us/step - accuracy: 0.7185 - loss: 0.5563
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 241us/step - accuracy: 0.8178 - loss: 0.4229
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 326us/step - accuracy: 0.8439 - loss: 0.3833
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 194us/step - accuracy: 0.8502 - loss: 0.3677
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 255us/step - accuracy: 0.8589 - loss: 0.3504
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 182us/step - accuracy: 0.8594 - loss: 0.3492
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 187us/step - accuracy: 0.8671 - loss: 0.3356
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 190us/step - accuracy: 0.8592 - loss: 0.3385
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 193us/step - accuracy: 0.8592 - loss: 0.3331
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 197us/step - accuracy: 0.8521 - loss: 0.3474
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 192us/step - accuracy: 0.8628 - loss: 0.3291
Epoch 12/100
250/250 ━━━━━━━━━━